# Chuẩn hóa dữ liệu

## 1. Giới thiệu

Tại thời điểm này, quy trình xử lý dữ liệu đã hoàn tất việc loại bỏ các tế bào không đạt chất lượng, các tạp nhiễm RNA từ môi trường, cũng như các tế bào doublet ra khỏi bộ dữ liệu. Kết quả thu được là một ma trận count, với các hàng tương ứng với tế bào và các cột tương ứng với gen. Những giá trị count này đại diện cho toàn bộ quá trình thí nghiệm scRNA-seq: từ bắt giữ phân tử, đến phiên mã ngược và cuối cùng là giải trình tự. Mỗi công đoạn trong quy trình đều có thể tạo ra độ biến thiên trong số lượng phân tử được ghi nhận, ngay cả khi xét trên cùng một loại tế bào. Do đó, những sai khác về mức độ biểu hiện gen giữa các tế bào trong ma trận count có thể chỉ là hệ quả của quá trình lấy mẫu chứ không phải do bản chất sinh học. Kết quả là, cả tập dữ liệu và ma trận count vẫn tồn tại những thành phần có phương sai lớn. Đây chính là thách thức chính trong quá trình phân tích, bởi phần lớn các phương pháp thống kê đều yêu cầu dữ liệu phải có phương sai tương đối đồng nhất.

Bước tiền xử lý "chuẩn hóa" nhằm điều chỉnh các số count thô trong tập dữ liệu để khắc phục ảnh hưởng của việc lấy mẫu không đồng đều, thông qua việc đưa phương sai quan sát được về một khoảng xác định. Trong thực tế, có nhiều kỹ thuật chuẩn hóa được sử dụng với các mức độ phức tạp khác nhau. Những phương pháp này thường được thiết kế sao cho phù hợp với các phân tích downstream và đáp ứng được các giả định thống kê cần thiết cho những phân tích đó.

Trong một nghiên cứu đánh giá gần đây được công bố bởi Ahlmann-Eltze và Huber [Ahlmann-Eltze và Huber, 2023], các tác giả đã so sánh 22 phương pháp biến đổi dữ liệu khác nhau cho dữ liệu tế bào đơn. Nghiên cứu này đánh giá hiệu quả của các kỹ thuật chuẩn hóa dựa trên mức độ trùng khớp giữa đồ thị tế bào với dữ liệu chuẩn. Tuy nhiên, cần lưu ý rằng vẫn chưa có một đánh giá toàn diện về tác động của các phương pháp chuẩn hóa này lên các bước phân tích hạ nguồn khác nhau. Do đó, chúng tôi khuyến nghị các nhà phân tích nên cân nhắc kỹ lưỡng trong việc lựa chọn phương pháp chuẩn hóa và luôn cần xem xét mục tiêu của các bước phân tích tiếp theo.

Chương này sẽ giới thiệu với độc giả ba phương pháp chuẩn hóa khác nhau: phép biến đổi logarit dịch chuyển, phương pháp chuẩn hóa scran, và phương pháp xấp xỉ giải tích của phần dư Pearson. Phép biến đổi logarit dịch chuyển có ưu điểm trong việc ổn định phương sai, phục vụ cho các bước giảm chiều dữ liệu và xác định các gen biểu hiện khác biệt. Phương pháp scran đã được kiểm nghiệm kỹ lưỡng và được sử dụng rộng rãi trong các tác vụ hiệu chỉnh batch. Trong khi đó, phương pháp phần dư Pearson giải tích đặc biệt phù hợp cho việc chọn lọc các gen có biến động sinh học và phát hiện các týp tế bào hiếm.

## 2. Phép biến đổi logarit dịch chuyển (Shifted logarithm)

Phương pháp chuẩn hóa đầu tiên chúng tôi giới thiệu là phép biến đổi logarit dịch chuyển, được xây dựng dựa trên phương pháp delta {cite}`dorfman1938note`. Phương pháp delta áp dụng một hàm phi tuyến $f(Y)$ lên các count thô $Y$ với mục đích làm cho phương sai trở nên đồng nhất hơn trong toàn bộ tập dữ liệu.

Phép biến đổi logarit dịch chuyển giải quyết vấn đề này thông qua công thức:

$$f(y) = \log(\frac{y}{s}+y_0)$$

trong đó $y$ là số count thô, $s$ là hệ số kích thước (size factor), và $y_0$ là giá trị count giả (pseudo-count). Hệ số kích thước được xác định cho từng tế bào nhằm điều chỉnh cho các biến động trong quá trình lấy mẫu và sự khác biệt về kích thước tế bào. Hệ số kích thước cho tế bào $c$ được tính theo công thức:

$$s_c = \frac{\sum_g y_{gc}}{L}$$

với $g$ là chỉ số của các gen khác nhau và $L$ là tổng mục tiêu. Có nhiều phương pháp khác nhau để xác định hệ số kích thước từ dữ liệu. Trong phần này, chúng tôi sẽ sử dụng giá trị mặc định của scanpy, trong đó $L$ là trung vị của độ sâu count thô trong tập dữ liệu. Nhiều mẫu phân tích sử dụng các giá trị cố định cho $L$, ví dụ như $L=10^5$ hoặc $L=10^6$, tạo ra các giá trị thường được gọi là số count trên một triệu (CPM). Đối với người mới bắt đầu, những giá trị này có thể có vẻ tùy ý, tuy nhiên việc sử dụng chúng có thể dẫn đến hiện tượng phân tán quá mức (overdispersion) lớn hơn nhiều so với thông thường trong các tập dữ liệu tế bào đơn.

Phép biến đổi logarit dịch chuyển là một kỹ thuật chuẩn hóa có tốc độ thực hiện nhanh, hiệu quả vượt trội so với các phương pháp khác trong việc khám phá cấu trúc tiềm ẩn của tập dữ liệu (đặc biệt khi kết hợp với phân tích thành phần chính), đồng thời có tác dụng tích cực trong việc ổn định phương sai cho các bước giảm chiều dữ liệu và xác định các gen biểu hiện khác biệt ở các giai đoạn tiếp theo.

Bạn có thể thực hành với trang sau:

[https://www.sc-best-practices.org/preprocessing_visualization/normalization.html#shifted-logarithm](https://www.sc-best-practices.org/preprocessing_visualization/normalization.html#shifted-logarithm)

## 3. Phương pháp chuẩn hóa scran

Phương pháp chuẩn hóa thứ hai, cũng dựa trên phương pháp delta, là phương pháp ước tính hệ số kích thước dựa trên nhóm của Scran. Scran tuân theo các nguyên tắc tương tự như phép biến đổi logarit dịch chuyển bằng cách tính $f(y) = \log(\frac{y}{s}+y_0)$ trong đó $y$ là số count thô, $s$ là hệ số kích thước và $y_0$ là giá trị count giả. Điểm khác biệt duy nhất là Scran sử dụng phương pháp giải chập (deconvolution) để ước tính các hệ số kích thước, dựa trên phép hồi quy tuyến tính trên các gen cho các nhóm tế bào. Phương pháp này nhằm mục đích xử lý tốt hơn sự khác biệt về độ sâu count giữa tất cả các tế bào có trong tập dữ liệu.

Các tế bào được phân chia thành các nhóm và Scran ước tính hệ số kích thước cho từng nhóm sử dụng phép hồi quy tuyến tính trên các gen. Scran đã được kiểm nghiệm kỹ lưỡng trong các tác vụ hiệu chỉnh batch và có thể dễ dàng được gọi thông qua package R tương ứng.

Bạn có thể thực hành với trang sau:

[https://www.sc-best-practices.org/preprocessing_visualization/normalization.html#shifted-logarithm](https://www.sc-best-practices.org/preprocessing_visualization/normalization.html#shifted-logarithm)

## 4. Phương pháp xấp xỉ giải tích của phần dư Pearson.

Kỹ thuật chuẩn hóa thứ ba được giới thiệu trong chương này là phương pháp xấp xỉ giải tích của phần dư Pearson. Phương pháp chuẩn hóa này được phát triển dựa trên nhận định rằng biến động giữa các tế bào trong dữ liệu scRNA-seq có thể là sự đan xen giữa tính không đồng nhất sinh học và các hiệu ứng kỹ thuật. Phương pháp này sử dụng phần dư Pearson từ 'hồi quy nhị thức âm điều hòa' để xây dựng mô hình nhiễu kỹ thuật trong dữ liệu. Đặc biệt, phương pháp này đưa độ sâu count (count depth) vào như một biến đồng biến trong mô hình tuyến tính tổng quát. Nghiên cứu của [Germain et al., 2020] trong một so sánh độc lập giữa các kỹ thuật chuẩn hóa khác nhau đã chỉ ra rằng phương pháp này loại bỏ được ảnh hưởng của việc lấy mẫu trong khi vẫn bảo toàn được tính không đồng nhất của tế bào trong tập dữ liệu. Đáng chú ý là phương pháp phần dư Pearson giải tích không đòi hỏi các bước thực nghiệm ở giai đoạn sau như thêm số count giả hay biến đổi logarit. Kết quả đầu ra của phương pháp này là các giá trị chuẩn hóa có thể dương hoặc âm. Phần dư âm đối với một tế bào và gen cho thấy số lượng count quan sát được ít hơn so với kỳ vọng khi so sánh với mức biểu hiện trung bình của gen và độ sâu giải trình tự của tế bào. Ngược lại, phần dư dương chỉ ra số lượng count nhiều hơn.

Bạn có thể thực hành với trang sau:

[https://www.sc-best-practices.org/preprocessing_visualization/normalization.html#shifted-logarithm](https://www.sc-best-practices.org/preprocessing_visualization/normalization.html#analytic-pearson-residuals)

## 5. Những điểm cần lưu ý

Dưới đây là những điểm quan trọng cần lưu ý về các phương pháp chuẩn hóa dữ liệu scRNA-seq:

*Mục đích của chuẩn hóa:*

- Điều chỉnh các số count thô để khắc phục ảnh hưởng của việc lấy mẫu không đồng đều
- Ổn định phương sai trong tập dữ liệu
- Tạo điều kiện thuận lợi cho các phân tích thống kê tiếp theo


*Phép biến đổi logarit dịch chuyển:*

- Tốc độ xử lý nhanh
- Hiệu quả trong việc phát hiện cấu trúc tiềm ẩn của dữ liệu
- Phù hợp cho giảm chiều dữ liệu và xác định gen biểu hiện khác biệt
- Sử dụng công thức $f(y) = \log(\frac{y}{s}+y_0)$
- Cần chú ý lựa chọn giá trị $L$ phù hợp để tránh hiện tượng phân tán quá mức


*Phương pháp Scran:*

- Dựa trên phương pháp delta như logarit dịch chuyển
- Sử dụng kỹ thuật giải chập để ước tính hệ số kích thước
- Phân tích theo nhóm tế bào
- Đặc biệt hiệu quả cho hiệu chỉnh batch
- Dễ dàng triển khai thông qua gói R


*Phương pháp phần dư Pearson giải tích:*

- Xử lý đồng thời cả biến động sinh học và hiệu ứng kỹ thuật
- Sử dụng hồi quy nhị thức âm điều hòa
- Không cần các bước hậu xử lý như thêm số count giả hay biến đổi logarit
- Kết quả có thể âm hoặc dương, phản ánh mức độ sai lệch so với kỳ vọng


*Lưu ý khi lựa chọn phương pháp:*

- Cân nhắc đặc điểm của tập dữ liệu
- Xem xét mục tiêu của các bước phân tích tiếp theo
- Đánh giá tác động của phương pháp chuẩn hóa lên kết quả phân tích
- Không có phương pháp nào hoàn hảo cho mọi tình huống


*Đánh giá hiệu quả:*

- Cần kiểm tra mức độ ổn định của phương sai sau chuẩn hóa
- Đánh giá khả năng bảo toàn thông tin sinh học
- Xem xét tính thực tiễn và hiệu quả tính toán
- Theo dõi tác động lên các bước phân tích downstream

Như vậy, chúng ta đã tìm hiểu ba phương pháp chuẩn hóa khác nhau cho dữ liệu scRNA-seq, mỗi phương pháp đều có những ưu điểm và ứng dụng riêng biệt. Phép biến đổi logarit dịch chuyển nổi bật với tốc độ thực hiện nhanh và hiệu quả trong việc ổn định phương sai, đặc biệt hữu ích cho các bước phân tích tiếp theo như giảm chiều dữ liệu. Phương pháp Scran với cách tiếp cận dựa trên nhóm đã chứng minh được hiệu quả trong việc hiệu chỉnh batch, trong khi phương pháp phần dư Pearson giải tích mang đến một cách tiếp cận toán học chặt chẽ để phân tách nhiễu kỹ thuật và đặc tính sinh học. Việc lựa chọn phương pháp chuẩn hóa phù hợp phụ thuộc vào đặc điểm của dữ liệu và mục tiêu phân tích cụ thể. Các nhà nghiên cứu nên cân nhắc kỹ lưỡng ưu và nhược điểm của từng phương pháp, đồng thời đánh giá tác động của chúng lên các bước phân tích tiếp theo để đưa ra quyết định phù hợp nhất cho nghiên cứu của mình.

## 6. Tham khảo

Bài này tổng hợp chủ yếu dựa trên [https://www.sc-best-practices.org/preprocessing_visualization/quality_control.html](https://www.sc-best-practices.org/preprocessing_visualization/normalization.html)

```{bibliography}
:filter: docname in docnames
:labelprefix: at
```

### 7.1. Tác giả/dịch giả
- Duy Pham

### 7.2. Người đánh giá